In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from substrate_helpers import *

In [5]:
import sqlite3
import re
import datetime
import pandas as pd

In [6]:
from database_queries import update_db

In [7]:
from substrateinterface import SubstrateInterface, Keypair
from substrateinterface.exceptions import SubstrateRequestException

substrate = SubstrateInterface(
    url="ws://127.0.0.1:9944",
    ss58_format=42,
    type_registry_preset='kusama'
)

In [8]:
con = sqlite3.connect('postthreadV1_write.db')
# con = sqlite3.connect('test1.db')
cur = con.cursor()

In [9]:
last_block_updated = update_db(start_block=0, backfill=True, schemaToUpdate=None)
con.commit()

post 102
comment 875
vote 1950
user 105
follow 1542


In [177]:

date_format = "%Y-%m-%d %H:%M:%S"

In [1]:
! cp postthreadV1_write.db postthreadV1_read.db

In [140]:
con.close()

In [10]:
    last_block = update_db(query_start=True,backfill=False, schemaToUpdate=None)

post 58


OperationalError: database is locked

In [185]:

    minutes_filter = 60*24
    date_where = ""
    if minutes_filter is not None:
        date_time = datetime.datetime.now() - datetime.timedelta(minutes=minutes_filter)
        date_str = date_time.strftime(date_format)
        date_where = f"post.date_minted >= '{date_str}'"

    order_by = ""
    if sort_by is None or sort_by == 'top':
        order_by = "ORDER BY total_votes DESC"
    if sort_by == 'new':
        order_by = "ORDER BY date_minted DESC"

    query = f'''
    SELECT * FROM post
    LEFT JOIN (
        SELECT parent_hash, SUM(num_votes) AS total_votes 
        FROM vote GROUP BY parent_hash
    ) votes ON votes.parent_hash = post.ipfs_hash
    LEFT JOIN (
        SELECT msa_id, username, password, profile_pic, wallet_ss58_address, block_number AS user_block_number, provider_key AS user_provider_key, date_minted AS user_date_minted
        FROM user u1
        WHERE date_minted = (SELECT max(date_minted) 
            FROM user u2
             WHERE u1.msa_id = u2.msa_id)
    ) users ON users.msa_id = post.msa_id_from_query
    WHERE {date_where}
    {order_by}
    '''
    df = pd.read_sql_query(query, con).drop_duplicates()
#     list(df.groupby('category')['total_votes'].count().sort_values(ascending=False).index)

In [190]:
datetime.datetime.now()

datetime.datetime(2022, 6, 29, 12, 13, 10, 110902)

In [188]:
!npx crust-cli publish /tmp/posts/Qmd491pRqjeiaUZznQQKqCB8K4Sb9QX2WwzWkWqiLXSQBw

Publish failed with: Error: invalid character '/' in '/tmp/Qmd491pRqjeiaUZznQQKqCB8K4Sb9QX2WwzWkWqiLXSQBw'


In [186]:
df

,category,title,body,url,is_nsfw,block_number,msa_id_from_query,provider_key,date_minted,ipfs_hash,parent_hash,total_votes,msa_id,username,password,profile_pic,wallet_ss58_address,user_block_number,user_provider_key,user_date_minted
0,politics,Trump lunged at Secret Service agent in rage w...,,https://www.cnbc.com/2022/06/28/trump-lunged-a...,0,17490,12429,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,2022-06-28 12:06:00,Qmd491pRqjeiaUZznQQKqCB8K4Sb9QX2WwzWkWqiLXSQBw,Qmd491pRqjeiaUZznQQKqCB8K4Sb9QX2WwzWkWqiLXSQBw,1804.0,12429,zossima,password,https://styles.redditmedia.com/t5_c44si/styles...,5GVnMeDZBbsFzWJrL9WEPZQcycL6aCXz7obfszDtZGdPBRbT,17492,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,2022-06-28 12:06:48
1,AnimalsBeingBros,That’s a good big brother,,https://v.redd.it/944st9iw6e891,0,17317,1245,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,2022-06-28 10:56:00,QmQyjLFKq73vGCWVZFCV9TVkw1jmhjSjJtPYAf2H1w8ZvZ,QmQyjLFKq73vGCWVZFCV9TVkw1jmhjSjJtPYAf2H1w8ZvZ,1662.0,1245,westcoastcdn19,password,https://styles.redditmedia.com/t5_2352hj/style...,5F3vLJkMdJ7erMkvmWSz1n9kkj5HEoCQJZPFWRyaGoWJsvTj,17317,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,2022-06-28 10:56:00
2,nba,[Charania] Brooklyn Nets star Kyrie Irving is ...,,https://twitter.com/ShamsCharania/status/15415...,0,14493,12331,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,2022-06-27 16:00:00,QmSbmTFgZduFccKLLr2N7S39BCb3xXjMfNowTnZPHZZWA4,QmSbmTFgZduFccKLLr2N7S39BCb3xXjMfNowTnZPHZZWA4,1452.0,12331,Tomheza,password,https://styles.redditmedia.com/t5_lrfcs/styles...,5H9SSHDDqhgjSNVwYUSnQS5ccf4iRhTHCjD2ogGe5WsqrYmu,16975,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,2022-06-28 08:39:12
3,nba,[Wojnarowski] John Wall is planning to sign wi...,,https://twitter.com/wojespn/status/15415986180...,0,14990,12331,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,2022-06-27 19:19:12,QmWzCf1ijgHxBVH3ZBxbpvRwUVS6iZLw118P9tY23GoPxk,QmWzCf1ijgHxBVH3ZBxbpvRwUVS6iZLw118P9tY23GoPxk,1090.0,12331,Tomheza,password,https://styles.redditmedia.com/t5_lrfcs/styles...,5H9SSHDDqhgjSNVwYUSnQS5ccf4iRhTHCjD2ogGe5WsqrYmu,16975,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,2022-06-28 08:39:12
4,nba,[Wojnarowski] ESPN Sources: Los Angeles Lakers...,,https://twitter.com/wojespn/status/15417971119...,0,16967,12331,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,2022-06-28 08:36:00,Qma4yoAJe7vns75TvzJvMzemJH2YNrEkSuieBXGbG2rcyD,Qma4yoAJe7vns75TvzJvMzemJH2YNrEkSuieBXGbG2rcyD,1027.0,12331,Tomheza,password,https://styles.redditmedia.com/t5_lrfcs/styles...,5H9SSHDDqhgjSNVwYUSnQS5ccf4iRhTHCjD2ogGe5WsqrYmu,16975,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,2022-06-28 08:39:12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2249,UpliftingNews,Proud Boys Showed Up To A Pride Month Library ...,,https://www.buzzfeednews.com/article/davidmack...,0,14663,741,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,2022-06-27 17:08:24,QmXi88SGsNADGc8vPbZM2ipupUA5q6Y6dqmkad5eZtaDGC,None,NaN,741,benfelix1,password,https://styles.redditmedia.com/t5_5ue646/style...,5EEcs1mP3Lrdayyf8h3R4KXPEo2beEujR9u3D42KrgP3gwPo,14663,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,2022-06-27 17:08:24
2250,OldSchoolCool,Angus Young. 1970❜s.,,https://i.redd.it/gsyehnocw8891.jpg,0,14663,2421,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,2022-06-27 17:08:24,QmambbghhjUGQSSbFLrt48SBJjxdZDqsYnExBExwEDgkuT,None,NaN,2421,frosty1965,password,https://styles.redditmedia.com/t5_x6tdv/styles...,5Fy2fYdUtxraF1VxwfdtzJbvco4JH7qGC6KHgzE4X2fZQdbm,14664,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,2022-06-27 17:08:48
2251,Smite,Mizkif is grateful to the Smite community,,https://clips.twitch.tv/DeliciousModernKangaro...,0,14663,12448,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,2022-06-27 17:08:24,QmPtndtjDvjh6JDiAZjc8WrNRYqFZFEkGT9JfoMfec4a8V,None,NaN,12448,Arantra,password,https://www.redditstatic.com/avatars/defaults/...,5GYsrCmg9Ao2tSZ9SD8EgPiZFQiqQtyz8A6EBDSB97p7stao,14663,5FHneW46xGXgs5mUiveU4sb

In [172]:
            query = f"""SELECT block_number FROM link ORDER BY date_minted DESC LIMIT 1"""
            start_block = int(pd.read_sql_query(query, con)['block_number'].iloc[0]) + 1

In [173]:
start_block

15781

In [171]:
pd.read_sql_query("""SELECT * FROM link""", con)

,account_type,account_value,block_number,msa_id_from_query,provider_key,date_minted
0,gmail,example@gmail.com,15780,382,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,2022-06-28 00:36:00


In [13]:
import networkx as nx

In [14]:
G = nx.Graph()

In [141]:

now = datetime.datetime.now()
tomorrow_time = (now + datetime.timedelta(days=1)).replace(hour=0, minute=0, second=0, microsecond=0)
(tomorrow_time - now).seconds

12431

In [142]:
datetime.datetime.now().strftime('%Y-%m-%d')

'2022-06-27'

In [139]:
(datetime.datetime.fromisoformat((datetime.datetime.now() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')) - datetime.datetime.now())

datetime.timedelta(seconds=12610, microseconds=906180)

In [140]:
date_time.seconds

AttributeError: 'datetime.datetime' object has no attribute 'seconds'

In [131]:
'2022-06-26 14:18:24' < date_time.strftime('%Y-%m-%d')

True

In [144]:
def get_user_level_df(user_msa_id=None):
    if user_msa_id is None:
        where_msa = ""
    else:
        where_msa = f"WHERE msa_id_from_query = {user_msa_id}"
        
    query = f'''
    SELECT u1.*, posts.count AS post_count, comments.count AS comment_count, votes.count AS vote_count, 
            follows.count AS follow_count
    FROM user u1
    LEFT JOIN (
        SELECT COUNT(*) AS count, msa_id_from_query 
        FROM post
        {where_msa}
        GROUP BY msa_id_from_query
    ) posts ON u1.msa_id_from_query = posts.msa_id_from_query 
    LEFT JOIN (
        SELECT COUNT(*) AS count, msa_id_from_query 
        FROM comment
        {where_msa}
        GROUP BY msa_id_from_query
    ) comments ON u1.msa_id_from_query = comments.msa_id_from_query
    LEFT JOIN (
        SELECT COUNT(*) AS count, msa_id_from_query 
        FROM vote
        {where_msa}
        GROUP BY msa_id_from_query
    ) votes ON u1.msa_id_from_query = votes.msa_id_from_query
    LEFT JOIN (
        SELECT COUNT(*) AS count, msa_id_from_query 
        FROM follow
        {where_msa}
        GROUP BY msa_id_from_query
    ) follows ON u1.msa_id_from_query = follows.msa_id_from_query
    WHERE {"" if where_msa == "" else 'u1.' + where_msa.split('WHERE ')[1] + ' AND'} date_minted = (
        SELECT max(date_minted) 
        FROM user u2
        WHERE u1.msa_id = u2.msa_id
    )
    '''
    return df

In [125]:
pd.read_sql_query("""SELECT block_number FROM post ORDER BY date_minted DESC LIMIT 1""", con)['block_number'].iloc[0]

181

In [147]:
df = get_user_level_df(382)

In [150]:
df.iloc[0].to_dict()

{'msa_id': 135,
 'username': 'Joescranium',
 'password': 'password',
 'profile_pic': 'https://styles.redditmedia.com/t5_dr6zo/styles/profileIcon_snooe1d9c1df-a961-4239-94a0-c5218f293e58-headshot-f.png?width=256&height=256&crop=256:256,smart&s=fec291c1877d252a6d8f2b1a7e0cfa0a40f51d1c',
 'wallet_ss58_address': '5FgXhnPwZP523WAtqvtcTKAytV76BGRgVNNxjm3rdUNDZYF2',
 'block_number': 3492,
 'msa_id_from_query': 135,
 'provider_key': '5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty',
 'date_minted': '2022-06-24 14:18:24',
 'post_count': 1,
 'comment_count': 10,
 'vote_count': 22,
 'follow_count': 1}

In [29]:

accounts = {'Alice': 693, 'Charlie': 382, 'Dave': 694, 'Eve': 695, 'Ferdie': 696}

In [35]:
post_hash

'QmbG2dxLWHKvdgsTqDzLjhD96BXrLQUeJkrLuasmPZKgn9'

In [162]:
    user_msa_id=None
    if user_msa_id is None:
        where_msa = ""
    else:
        where_msa = f"WHERE msa_id_from_query = {user_msa_id}"
        
    query = f'''
    SELECT u1.*, posts.count AS post_count, comments.count AS comment_count, votes.count AS vote_count, 
            follows.count AS follow_count
    FROM user u1
    LEFT JOIN (
        SELECT COUNT(*) AS count, msa_id_from_query 
        FROM post
        {where_msa}
        GROUP BY msa_id_from_query
    ) posts ON u1.msa_id_from_query = posts.msa_id_from_query 
    LEFT JOIN (
        SELECT COUNT(*) AS count, msa_id_from_query 
        FROM comment
        {where_msa}
        GROUP BY msa_id_from_query
    ) comments ON u1.msa_id_from_query = comments.msa_id_from_query
    LEFT JOIN (
        SELECT COUNT(*) AS count, msa_id_from_query 
        FROM vote
        {where_msa}
        GROUP BY msa_id_from_query
    ) votes ON u1.msa_id_from_query = votes.msa_id_from_query
    LEFT JOIN (
        SELECT COUNT(*) AS count, msa_id_from_query 
        FROM follow
        {where_msa}
        GROUP BY msa_id_from_query
    ) follows ON u1.msa_id_from_query = follows.msa_id_from_query
    WHERE {"" if where_msa == "" else 'u1.' + where_msa.split('WHERE ')[1] + ' AND'} date_minted = (
        SELECT max(date_minted) 
        FROM user u2
        WHERE u1.msa_id = u2.msa_id
    )
    '''
    df = pd.read_sql_query(query, con)
    df

,msa_id,username,password,profile_pic,wallet_ss58_address,block_number,msa_id_from_query,provider_key,date_minted,post_count,comment_count,vote_count,follow_count
0,21,cris12021202,password,https://styles.redditmedia.com/t5_229lns/style...,5FNzPtPJSdHWr82L81WBtYQgs2HXdPswwJZx1S8Z8MNVadZX,181,21,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,2022-06-23 16:11:18,NaN,NaN,2.0,94.0
1,24,jago-delgado,password,https://styles.redditmedia.com/t5_6ecy5z/style...,5Ct3Z3X2goXx22GBXnuQYWg7zUtnN52MyRcD8rjtd35YqRpR,181,24,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,2022-06-23 16:11:18,NaN,NaN,2.0,95.0
2,31,syrianrefugee02,password,https://styles.redditmedia.com/t5_5t7sb/styles...,5Hn6XqgSyNQiTyRgzRPXc4b1AEN6ZsVWx9fDmoG8ZvT52Yrk,181,31,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,2022-06-23 16:11:18,NaN,NaN,2.0,57.0
3,45,thethirdidiot,password,https://styles.redditmedia.com/t5_93jkx/styles...,5GHjh5n1VFHdWPKQUm74n3LqVtYB69UsZVodxsnW54PbgjYp,181,45,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,2022-06-23 16:11:18,NaN,NaN,2.0,NaN
4,56,SusSauceIsSus,password,https://styles.redditmedia.com/t5_5i1vva/style...,5HWDS3rSyNyDiiE4rKvJvJyR8y3RjUxq2WvmNsxqeE6xzr66,181,56,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,2022-06-23 16:11:18,NaN,NaN,2.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8923,1672,arjun667,password,https://styles.redditmedia.com/t5_6dzgfh/style...,5FR7y44Ns3CUGX6xyQkh8G7xDESNCzdCZzkLt3tUxycsX3LQ,15314,1672,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,2022-06-27 21:29:12,1.0,2.0,6.0,1.0
8924,12746,_mad_eye_,password,https://styles.redditmedia.com/t5_w52kc/styles...,5CAgfaym6zzXvZ6WSF6vqnoZM8W6XuijPqpV5eXG69WKBYa8,15314,12746,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,2022-06-27 21:29:12,1.0,5.0,12.0,1.0
8925,6625,Chilly-777,password,https://styles.redditmedia.com/t5_4qus3v/style...,5Di3Sj2XViCiU8vUyj2X1TYFah1Y4fWDs7gMQBQw4W3fditS,15315,6625,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,2022-06-27 21:29:42,2.0,11.0,26.0,2.0
8926,12747,jaydno,password,https://styles.redditmedia.com/t5_qgzd7/styles...,5DHpxcQuJz1FWdtsApKbNwpBDKaH1oHbiUXgtNDgpcnm8Zzr,15315,12747,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,2022-06-27 21:29:42,1.0,3.0,8.0,1.0


In [168]:
df.sort_values("post_count", ascending=False)

,msa_id,username,password,profile_pic,wallet_ss58_address,block_number,msa_id_from_query,provider_key,date_minted,post_count,comment_count,vote_count,follow_count
8368,90,regian24,password,https://styles.redditmedia.com/t5_s34hp/styles...,5Ei5uBvWkwoXwL1dX482NSaZkKdGnB6CBFbgsyu5YfcvZxgk,14494,90,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,2022-06-27 16:00:24,36.0,140.0,363.0,30.0
8882,1210,suckontits,password,https://styles.redditmedia.com/t5_62g0eg/style...,5DMLi9Epaazk12h7He4BBCkTxTqqqmYXTakrxpiJPgpNLmcs,15312,1210,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,2022-06-27 21:28:24,34.0,116.0,288.0,24.0
8740,22,SnooCupcakes8607,password,https://styles.redditmedia.com/t5_2ua6ec/style...,5DcfcMgQrJ6QyCVj88WV5amxpeUAMwQtFFE7R1dWQ667Fxht,15158,22,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,2022-06-27 20:26:24,27.0,151.0,376.0,210.0
8136,1849,MantasChan,password,https://styles.redditmedia.com/t5_4c8o7v/style...,5GYMTMsVwrjWTwDy5eCy3Xd6KK2ANwm5h8Kbw2KWXUWYx7zX,14127,1849,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,2022-06-27 13:32:48,26.0,105.0,262.0,25.0
8869,68,Abhirup_0,password,https://styles.redditmedia.com/t5_5ygx5i/style...,5FByRpDAnDpFt11emkmvz1VYw5Eaadcnm2f3tu1ReZX7Ziqw,15311,68,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,2022-06-27 21:28:00,22.0,142.0,357.0,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7774,11868,gaudior040618,password,https://styles.redditmedia.com/t5_y4fr9/styles...,5GzcVse9FXHdtAWEdwbwnPSWWR6TFzKciSibcNy9YSHzxhtD,12748,11868,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,2022-06-27 04:19:42,NaN,3.0,8.0,1.0
8827,693,Alice,password,profile_pic,5CrGbnxQvf7jkBbKYoA2aNB4E4CpVkEBtjoQW6JaPzLq91FA,15246,693,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,2022-06-27 21:01:42,NaN,NaN,NaN,4.0
8829,694,Dave,password,profile_pic,5C5echHKJwNfWE3AE7rk1N8axLo7fBqFYYBL1xZ6ohn1oiZt,15246,694,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,2022-06-27 21:01:42,NaN,NaN,NaN,5.0
8830,695,Eve,password,profile_pic,5H5o6fAGUycA8SKMTtRAeTpr6o93BU5fmAg2VrhPRntezV21,15246,695,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,2022-06-27 21:01:42,NaN,NaN,NaN,5.0


In [155]:
import time
time.time()

1656392501.3863804

In [118]:
df[df['post_count'] > 0]

,msa_id,username,password,profile_pic,wallet_ss58_address,block_number,msa_id_from_query,provider_key,date_minted,post_count,comment_count,vote_count,follow_count
0,135,Joescranium,password,https://styles.redditmedia.com/t5_dr6zo/styles...,5FgXhnPwZP523WAtqvtcTKAytV76BGRgVNNxjm3rdUNDZYF2,3492,135,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,2022-06-24 14:18:24,1,10,22,1


In [161]:
query = """
    SELECT * FROM post 
    LEFT JOIN (
        SELECT parent_hash, SUM(num_votes) AS total_votes 
        FROM vote GROUP BY parent_hash
    ) votes ON votes.parent_hash = post.ipfs_hash
    LEFT JOIN (
        SELECT msa_id, username, password, profile_pic, wallet_ss58_address, block_number AS user_block_number, provider_key AS user_provider_key, date_minted AS user_date_minted
        FROM user u1
        WHERE date_minted = (SELECT max(date_minted) 
            FROM user u2
             WHERE u1.msa_id = u2.msa_id)
    ) users ON users.msa_id = post.msa_id_from_query
    WHERE post.ipfs_hash = 'QmbG2dxLWHKvdgsTqDzLjhD96BXrLQUeJkrLuasmPZKgn9'
    ORDER BY total_votes DESC
    """
df = pd.read_sql_query(query, con)
df

,category,title,body,url,is_nsfw,block_number,msa_id_from_query,provider_key,date_minted,ipfs_hash,parent_hash,total_votes,msa_id,username,password,profile_pic,wallet_ss58_address,user_block_number,user_provider_key,user_date_minted
0,test,test title,test post,,0,14252,382,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,2022-06-27 14:22:48,QmbG2dxLWHKvdgsTqDzLjhD96BXrLQUeJkrLuasmPZKgn9,None,None,382,Charlie,password,profile_pic,5FqWBycSL7R7bG3VNRRHftv98kCjGCQ3oPT34tMUTwfsvcUj,15246,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,2022-06-27 21:01:42


In [96]:
query = '''
SELECT * 
FROM post
'''
df = pd.read_sql_query(query, con)
df

,category,title,body,url,is_nsfw,block_number,msa_id_from_query,provider_key,date_minted,ipfs_hash
0,MadeMeSmile,He’s going to beat you someday …,,https://i.redd.it/g6y9ofdqig791.jpg,0,3490,97,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,2022-06-24 14:17:42,QmfNRzoKbqXcC7JwZUrgFdSs7VTNid36kuLxCyySyDPznL
1,MadeMeSmile,He’s going to beat you someday …,,https://i.redd.it/g6y9ofdqig791.jpg,0,3491,97,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,2022-06-24 14:18:00,QmfNRzoKbqXcC7JwZUrgFdSs7VTNid36kuLxCyySyDPznL
2,MadeMeSmile,He’s going to beat you someday …,,https://i.redd.it/g6y9ofdqig791.jpg,0,3492,97,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,2022-06-24 14:18:24,QmfNRzoKbqXcC7JwZUrgFdSs7VTNid36kuLxCyySyDPznL
3,news,Supreme Court overturns Roe v. Wade; states ca...,,https://apnews.com/article/854f60302f21c2c3512...,0,3492,135,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,2022-06-24 14:18:24,QmQYNzt5QcrJ9igx4o7jwtmYZcHyqrHvtSGoRo52yA9vdo
4,interestingasfuck,Anonychia is the partial or total absence of o...,,https://i.redd.it/c3cxommd2g791.jpg,0,3492,136,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,2022-06-24 14:18:24,QmYGb6jr1n6nLmJvK1PPG2ZH8KMfDG9PJgmr5nSvqmyYuq
...,...,...,...,...,...,...,...,...,...,...
10979,PoliticalHumor,"Great idea, but pick something that affects them.",,https://i.redd.it/qmy9dsm4ra891.jpg,0,14834,12541,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,2022-06-27 18:16:48,Qmd5P1mw7HqE2Hnby7iYogVg3k4qbVKyfH4CWvfFuN31RJ
10980,RATS,Babies enjoying their salad!! (Grey in the las...,,https://www.reddit.com/gallery/vm9p1r,0,14834,12542,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,2022-06-27 18:16:48,QmUTaV2SitNHBiwG5oCKnB89CEXAmFQdf9j8wT1dZWSaL7
10981,LateStageCapitalism,oligarchs keep democrats around to make us fee...,,https://i.redd.it/8c8e71ahb9891.jpg,0,14834,12543,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,2022-06-27 18:16:48,QmVw6zj9ykA9BvpeWv1qAu3daLKrJZknjXKF2fCvw3wJj2
10982,worldnews,Abandoning God: Christianity plummets as ‘non-...,,https://www.smh.com.au/national/abandoning-god...,0,14834,12544,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,2022-06-27 18:16:48,QmWcfzFJ4dohEZz3RVbdrw43eS99jAYeTveaQS2nhP8V1c


In [133]:
query = '''
SELECT * 
FROM follow
WHERE protagonist_msa_id = 1335 
ORDER BY protagonist_msa_id
LIMIT 10
'''
df = pd.read_sql_query(query, con)
df

,protagonist_msa_id,antagonist_msa_id,event,block_number,msa_id_from_query,signer,date_minted
0,1335,1337,follow,11168,1335,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-22-2022 20:46:54
1,1335,1337,follow,11169,1335,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-22-2022 20:47:12
2,1335,1339,follow,10971,1335,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-22-2022 19:28:00
3,1335,1339,follow,11169,1335,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-22-2022 20:47:12
4,1335,1339,unfollow,10975,1335,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-22-2022 19:29:36
5,1335,1341,follow,11169,1335,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-22-2022 20:47:12
6,1335,1343,follow,11169,1335,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-22-2022 20:47:12


In [183]:
user_msa_id = 1355
query = f'''
SELECT *
FROM post
'''
df = pd.read_sql_query(query, con)
df

,category,username,profile_pic,title,body,url,is_nsfw,block_number,msa_id_from_query,signer,date_minted,ipfs_hash
0,FunnyAnimals,pietradolce,https://styles.redditmedia.com/t5_2hkan3/style...,Lmao,,https://i.redd.it/u73prwqx4z691.png,0,6968,1,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 16:44:24,Qmb2LJ5pCQKa3jDGwZxUucSXpjmh4qTuWR3NkmEuQ5bxJV
1,antiwork,caIyps0o,https://styles.redditmedia.com/t5_59xly6/style...,companies be like:,,https://i.redd.it/ciyfgkyx0z691.jpg,0,7000,182,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 16:57:12,QmNQmmH1Kpbdtt5WaDchgfuHyRJj7rJXqJerC6Wdtem8UJ
2,todayilearned,filondo,https://styles.redditmedia.com/t5_5orduo/style...,TIL Wells Fargo was forced by the feds to rehi...,,https://money.cnn.com/2017/04/03/news/companie...,0,7030,184,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 17:09:12,QmQtJt9yYhFTDhThW4kuCBUaYzV4VGwi5JzBSLKPSSi6vx
3,MadeMeSmile,pietradolce,https://styles.redditmedia.com/t5_2hkan3/style...,Meme in real life,,https://i.redd.it/4q05968b60791.jpg,0,7046,1,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 17:16:00,QmaEKD9igfwiDNMVufvXTSQ3SDorSna1fXBzhpquYGdZwT
4,MadeMeSmile,effineuphoria,https://styles.redditmedia.com/t5_5tf9d7/style...,This is the most heartwarming rejection email ...,,https://i.redd.it/oa2jssgt1y691.jpg,0,7066,187,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 17:24:00,QmW7TQJpFASYN9XVDmyiW4UcEBmvJ3XpZm5nRabKsnZwFo
...,...,...,...,...,...,...,...,...,...,...,...,...
133,news,CrotalusHorridus,https://www.redditstatic.com/avatars/defaults/...,Uvalde City Council denies Pete Arredondo❜s le...,,https://abcnews.go.com/US/uvalde-city-council-...,0,10993,1485,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-22-2022 19:36:54,QmYtqVUtS817QiVpQyVK8rhCzX1ZhUCe9gorWnq1ja8MMB
134,AnimalCrossing,ladyburgerandcatnap,https://styles.redditmedia.com/t5_69ihsy/style...,If only...,,https://i.redd.it/grhckw9d56791.jpg,0,11004,1495,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-22-2022 19:41:12,QmUanMERUpSf5WRjXDv6AtLZER4ZynV3DVZfdopFf5GWpJ
135,Eldenring,nbshar,https://styles.redditmedia.com/t5_6x6dw/styles...,We made Elden Ring look like it❜s from 1995 an...,,https://v.redd.it/w4l2etayv6791,0,11015,1506,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-22-2022 19:45:36,QmRaFbimRRjTiHbaXA4Y1YMD2YiPJtFVaSFZgEfv7v3dCQ
136,Unexpected,General_Degenerate_,https://styles.redditmedia.com/t5_2z6p4d/style...,Dog tries to catch the ball,,https://v.redd.it/5gmsvo9w15791,0,11026,1516,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-22-2022 19:50:00,QmUdrapFnJ5yVhdKUJaxnTR9oyowzZujuABEaXvFaV1CXX


In [184]:
user_msa_id = 1355
query = f'''
SELECT user.*, posts.count AS post_count, comments.count AS comment_count, votes.count AS vote_count, 
        follows.count AS follow_count
FROM user
LEFT JOIN (
    SELECT COUNT(*) AS count, msa_id_from_query 
    FROM post
    WHERE msa_id_from_query = {user_msa_id}
) posts ON user.msa_id_from_query = posts.msa_id_from_query 
LEFT JOIN (
    SELECT COUNT(*) AS count, msa_id_from_query 
    FROM comment
    WHERE msa_id_from_query = {user_msa_id}
) comments ON user.msa_id_from_query = comments.msa_id_from_query
LEFT JOIN (
    SELECT COUNT(*) AS count, msa_id_from_query 
    FROM vote
    WHERE msa_id_from_query = {user_msa_id}
) votes ON user.msa_id_from_query = votes.msa_id_from_query
LEFT JOIN (
    SELECT COUNT(*) AS count, msa_id_from_query 
    FROM follow
    WHERE msa_id_from_query = {user_msa_id}
) follows ON user.msa_id_from_query = follows.msa_id_from_query
WHERE user.msa_id_from_query = {user_msa_id} 
'''
df = pd.read_sql_query(query, con).fillna(0)
df

,msa_id,username,password,profile_pic,wallet_ss58_address,block_number,msa_id_from_query,signer,date_minted,post_count,comment_count,vote_count,follow_count
0,1355,hickgorilla,password,https://www.redditstatic.com/avatars/defaults/...,5G9NjTUpK6ZQaiukJmhKnBVzWnESHFpWUFjCfRXoGwkTHXQ5,10586,1355,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-22-2022 16:53:36,None,1,2,5


In [79]:
query = '''
SELECT * FROM vote
'''
df = pd.read_sql_query(query, con)
df

,post_hash,parent_hash,parent_type,num_votes,block_number,msa_id_from_query,signer,date_minted
0,QmaEKD9igfwiDNMVufvXTSQ3SDorSna1fXBzhpquYGdZwT,QmaEKD9igfwiDNMVufvXTSQ3SDorSna1fXBzhpquYGdZwT,post,72120,7046,1,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 17:16:00
1,QmaEKD9igfwiDNMVufvXTSQ3SDorSna1fXBzhpquYGdZwT,QmaEKD9igfwiDNMVufvXTSQ3SDorSna1fXBzhpquYGdZwT,post,0,7046,1,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 17:16:00
2,QmW7TQJpFASYN9XVDmyiW4UcEBmvJ3XpZm5nRabKsnZwFo,QmW7TQJpFASYN9XVDmyiW4UcEBmvJ3XpZm5nRabKsnZwFo,post,68123,7066,187,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 17:24:00
3,QmW7TQJpFASYN9XVDmyiW4UcEBmvJ3XpZm5nRabKsnZwFo,QmW7TQJpFASYN9XVDmyiW4UcEBmvJ3XpZm5nRabKsnZwFo,post,0,7066,187,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 17:24:00
4,QmX2Ny3mMA4pLZiW2XnrNTFda7iHgQtgq46TbaPmpXF25G,QmX2Ny3mMA4pLZiW2XnrNTFda7iHgQtgq46TbaPmpXF25G,post,68130,7117,188,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 17:44:24
...,...,...,...,...,...,...,...,...
993,QmVGe28n9JU6m7uLLiHKiLwkG6dXeUUUHV2tCzPS7tRwfS,QmT48JN2G11MZ6B8tNekZ4dMAiRsfrPTsWiuedTqsCvLy4,comment,0,7635,658,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 21:11:36
994,QmVGe28n9JU6m7uLLiHKiLwkG6dXeUUUHV2tCzPS7tRwfS,QmVvRMUz8oGggXwk6sgJCJjqVV4wrzyaPDg1NBsfFzURh7,comment,533,7636,659,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 21:12:00
995,QmVGe28n9JU6m7uLLiHKiLwkG6dXeUUUHV2tCzPS7tRwfS,QmVvRMUz8oGggXwk6sgJCJjqVV4wrzyaPDg1NBsfFzURh7,comment,0,7636,659,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 21:12:00
996,QmVGe28n9JU6m7uLLiHKiLwkG6dXeUUUHV2tCzPS7tRwfS,QmZe9nSLRFuhE5fvxK6BWPSpycYPmhBpJRWeiNCvpwfBT9,comment,1546,7637,660,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 21:12:24


In [152]:
query = '''
SELECT * FROM user
'''
df = pd.read_sql_query(query, con)
df

,msa_id,username,password,profile_pic,wallet_ss58_address,block_number,msa_id_from_query,signer,date_minted
0,438,Mental_Shelter,password,https://www.redditstatic.com/avatars/defaults/...,5DUemZiN3jQKfaedGo1bEQKRnercMrgAHianX21hkrcChEuC,7397,438,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 19:36:24
1,439,dimechimes,password,https://www.redditstatic.com/avatars/defaults/...,5E741P4DGM94ozrLjrLpNJe9HGPgT5TCAQs1yPbYvbALRDMZ,7398,439,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 19:36:54
2,440,Sl0ppy0tter,password,https://styles.redditmedia.com/t5_52wkdg/style...,5EFK76PX3ho6qGVYax1z3mEyHYxYKJxpAE4M1mV12NgSPAXn,7399,440,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 19:37:12
3,441,mightylordredbeard,password,https://www.redditstatic.com/avatars/defaults/...,5GKV2pAsjGJUKJ9wEW4tF4yxL2ogtUvox8Gyejdd4wHa5VAi,7400,441,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 19:37:36
4,442,human_male_123,password,https://www.redditstatic.com/avatars/defaults/...,5GBg27doAGAJasPjcbNvrbGrMDvkYBxM2CY2CFQACye7UGr7,7401,442,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 19:38:00
...,...,...,...,...,...,...,...,...,...
302,740,brigrrrl,password,https://styles.redditmedia.com/t5_28rzsh/style...,5CotCQUjesc6BnWt6x7YyersypyaCuswKoVPqXsjDXtc3wH3,7722,740,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 21:46:24
303,741,eduardtee,password,https://styles.redditmedia.com/t5_6msk8/styles...,5CLxfPDYkXFUnGYzJFPkQiKdXVetR81UcxqnfygmwpwtFJuG,7723,741,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 21:46:54
304,742,Victory1871,password,https://www.redditstatic.com/avatars/defaults/...,5EFmBd3PDykbXC1YfQXxGbndgR9nJubokV3BENTB6TXva4Hx,7725,742,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 21:47:36
305,743,Stringbean1109,password,https://styles.redditmedia.com/t5_5uhgku/style...,5GmuriRZFWdMhQhU1mN8t1nzMvcKcvWd5B9bKzTVFz1zLooi,7726,743,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 21:48:00


In [156]:
import uvicorn
uvicorn.server.Config?

In [153]:
query = '''
SELECT * FROM comment
'''
df = pd.read_sql_query(query, con)
df

,post_hash,parent_hash,parent_type,username,profile_pic,body,block_number,msa_id_from_query,signer,date_minted,ipfs_hash
0,QmX2Ny3mMA4pLZiW2XnrNTFda7iHgQtgq46TbaPmpXF25G,QmX2Ny3mMA4pLZiW2XnrNTFda7iHgQtgq46TbaPmpXF25G,post,MedicatedAxeBot,https://www.redditstatic.com/avatars/defaults/...,Dank[.](https://i.imgur.com/3bQtuMO.png)\n\n--...,7118,189,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 17:44:54,QmSUp3SQeLvojwNfxYFj6DUDZCo44iKDfwJT3xyHGEzkFh
1,QmX2Ny3mMA4pLZiW2XnrNTFda7iHgQtgq46TbaPmpXF25G,QmX2Ny3mMA4pLZiW2XnrNTFda7iHgQtgq46TbaPmpXF25G,post,Vendis123,https://styles.redditmedia.com/t5_qr7mm/styles...,"Well, he doesn❜t write his own speeches",7119,190,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 17:45:12,QmdGPH6RK7Pj31NEtkrECKzMvQBksYyEiv9XQGc7K2dFHX
2,QmX2Ny3mMA4pLZiW2XnrNTFda7iHgQtgq46TbaPmpXF25G,QmX2Ny3mMA4pLZiW2XnrNTFda7iHgQtgq46TbaPmpXF25G,post,Dubr1s,https://styles.redditmedia.com/t5_1nasz1/style...,And it❜s most upvoted comment with a lot of aw...,7120,191,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 17:45:36,QmfDvg9i9RqbPCPemgnDdPR6HsG2YFeY4dZinR9QVQ3iky
3,QmX2Ny3mMA4pLZiW2XnrNTFda7iHgQtgq46TbaPmpXF25G,QmX2Ny3mMA4pLZiW2XnrNTFda7iHgQtgq46TbaPmpXF25G,post,iron_fan_man,https://styles.redditmedia.com/t5_38wc0r/style...,YES he’s like bob the banana,7121,192,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 17:46:00,QmeJxRNR92e7DVpEpZKfstRnkDCkjywGsfrPvuZPcBKe9D
4,QmX2Ny3mMA4pLZiW2XnrNTFda7iHgQtgq46TbaPmpXF25G,QmX2Ny3mMA4pLZiW2XnrNTFda7iHgQtgq46TbaPmpXF25G,post,Probablyaretweetbot,https://styles.redditmedia.com/t5_4rw7la/style...,u/deleted yo,7122,193,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 17:46:24,QmZTmoZSaCypPbXkfDYdHfSA2UAnVPBhzx2VAYD5e3cAFB
...,...,...,...,...,...,...,...,...,...,...,...
527,QmfGyt1GNdPBy5b1WbXGi6PBtsSmtDwtGiNjXg6BEXXXGh,QmfGyt1GNdPBy5b1WbXGi6PBtsSmtDwtGiNjXg6BEXXXGh,post,brigrrrl,https://styles.redditmedia.com/t5_28rzsh/style...,I love this. Reminds me of my childhood a bit....,7722,740,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 21:46:24,QmNWqPDQ58GuazdSLyuaCV7E3w7hDgvrcBUXunKUytt154
528,QmfGyt1GNdPBy5b1WbXGi6PBtsSmtDwtGiNjXg6BEXXXGh,QmfGyt1GNdPBy5b1WbXGi6PBtsSmtDwtGiNjXg6BEXXXGh,post,eduardtee,https://styles.redditmedia.com/t5_6msk8/styles...,So sweet. Your parents deserve a hug for this ...,7723,741,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 21:46:54,QmdV5r1NDRwGe5ZieEEtSVhJtDETXwQ22ZD3pAxEafvVYz
529,QmfGyt1GNdPBy5b1WbXGi6PBtsSmtDwtGiNjXg6BEXXXGh,QmfGyt1GNdPBy5b1WbXGi6PBtsSmtDwtGiNjXg6BEXXXGh,post,unclejarjarbinks,https://styles.redditmedia.com/t5_180224/style...,Best part was you didn❜t have to deal with the...,7724,1,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 21:47:12,QmW8Zg4cqFGH9XjLYTL3er6W4VCBw2RRxmCaRP8SFsU49n
530,QmfGyt1GNdPBy5b1WbXGi6PBtsSmtDwtGiNjXg6BEXXXGh,QmfGyt1GNdPBy5b1WbXGi6PBtsSmtDwtGiNjXg6BEXXXGh,post,Victory1871,https://www.redditstatic.com/avatars/defaults/...,Wow that’s wholesome. This just made my day.,7725,742,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 21:47:36,QmX9XL7kS92WfgV9AjYDX11BaWakLpqr8FGqpazRQyL6s8


In [61]:
query = '''
SELECT DISTINCT(category) FROM post
'''
df = pd.read_sql_query(query, con)
df

,category
0,AbruptChaos
1,AbsoluteUnits
2,AnimalCrossing
3,AnimalsBeingBros
4,AnimalsBeingDerps
...,...
64,therewasanattempt
65,todayilearned
66,tumblr
67,wholesomememes


In [60]:
    page_number = 1
    num_posts = 10
    category = None
    sort_by = "top"
    minutes_filter = None
    user_msa_id = 1


    if minutes_filter is None:
        minutes_filter = 60*24*100

    category_where = ""
    if category:
        category_where = f"AND post.category = '{category}'"

    user_where = ""
    if user_msa_id:
        user_where = f"AND post.msa_id_from_query = {user_msa_id}"

    order_by = ""
    if sort_by is None or sort_by == 'top':
        order_by = "ORDER BY total_votes DESC"
    if sort_by == 'new':
        order_by = "ORDER BY date_minted DESC"

    lower_bound = (page_number - 1) * num_posts
    upper_bound = page_number * num_posts

    date_format = "%m-%d-%Y %H:%M:%S"
#     date_format = "%Y-%m-%d %H:%M:%S"
    date_time = datetime.datetime.now() - datetime.timedelta(minutes=minutes_filter)
    date_str = date_time.strftime(date_format)

    query = f'''
    SELECT * FROM post 
    JOIN (
        SELECT parent_hash, SUM(num_votes) AS total_votes 
        FROM vote GROUP BY parent_hash
    ) votes ON votes.parent_hash = post.ipfs_hash
    WHERE post.date_minted >= '{date_str}' {category_where} {user_where}
    {order_by}
    LIMIT {lower_bound}, {upper_bound}
    '''
    df = pd.read_sql_query(query, con)
    df

,category,username,profile_pic,title,body,url,is_nsfw,block_number,msa_id_from_query,signer,date_minted,ipfs_hash,parent_hash,total_votes
0,MadeMeSmile,Mapegz,https://styles.redditmedia.com/t5_387jy7/style...,Ronaldo is a Classy Madlad,,https://i.redd.it/7pqlch1io5791.jpg,0,10273,1,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-22-2022 14:48:24,QmRGavXQG3ARiu5RTcfRGsJ8Rc4hic51Z13xpBMQhyqY9D,QmRGavXQG3ARiu5RTcfRGsJ8Rc4hic51Z13xpBMQhyqY9D,100137
1,Unexpected,deadstar420,https://styles.redditmedia.com/t5_ye8a4/styles...,That’s some skillful driving,,https://v.redd.it/ih8tuai2b6791,0,10284,1,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-22-2022 14:52:54,QmaqgCLChopJFkQ8hjhgUMqo18eiNVqAT3N3HVqRsws84J,QmaqgCLChopJFkQ8hjhgUMqo18eiNVqAT3N3HVqRsws84J,98517
2,MadeMeSmile,pietradolce,https://styles.redditmedia.com/t5_2hkan3/style...,Meme in real life,,https://i.redd.it/4q05968b60791.jpg,0,7046,1,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 17:16:00,QmaEKD9igfwiDNMVufvXTSQ3SDorSna1fXBzhpquYGdZwT,QmaEKD9igfwiDNMVufvXTSQ3SDorSna1fXBzhpquYGdZwT,72120
3,MadeMeSmile,GroundbreakingSet187,https://styles.redditmedia.com/t5_316hy3/style...,Onlygrans would be so awesome …,,https://i.redd.it/an25bko1o2791.jpg,0,10339,1,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-22-2022 15:14:54,QmUy6DfUT5TjL9Q84zPkVFY8XHR9WpCYuxRZQNcXadz2g6,QmUy6DfUT5TjL9Q84zPkVFY8XHR9WpCYuxRZQNcXadz2g6,70590
4,oddlysatisfying,Mapegz,https://styles.redditmedia.com/t5_387jy7/style...,The way they prepare Spaghetti,,https://v.redd.it/uvwzdfl8m6791,0,10424,1,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-22-2022 15:48:54,QmaJ9y4rjbjogK8DwCz6NJa8TyLsrr4jrvUXi7jLe8JQmw,QmaJ9y4rjbjogK8DwCz6NJa8TyLsrr4jrvUXi7jLe8JQmw,68085
5,PublicFreakout,TheBadMighty,https://styles.redditmedia.com/t5_328wsl/style...,Teen Choked By Police Who Entered His Home Wit...,,https://v.redd.it/q7n09kyggy691,0,7139,1,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 17:53:12,QmRg8oUUGTpKczKj3U8QkoJ3NZsUrQPw2zXqppkpeLyEet,QmRg8oUUGTpKczKj3U8QkoJ3NZsUrQPw2zXqppkpeLyEet,67763
6,PoliticalHumor,DaFunkJunkie,https://styles.redditmedia.com/t5_22tglj/style...,When a therapist gets you,,https://i.redd.it/72n1ldcxyy691.jpg,0,7150,1,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 17:57:36,QmNsApjrk2m9ymM1fTmPFrmEv9MZsnwnNXWBZLDF8UVPMD,QmNsApjrk2m9ymM1fTmPFrmEv9MZsnwnNXWBZLDF8UVPMD,64768
7,interestingasfuck,Real_Prophet01,https://styles.redditmedia.com/t5_6dmhau/style...,Zero to hero before & after animation,,https://v.redd.it/jlmw0wvqsw691,0,7183,1,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 18:10:54,QmWJWbHKkKBfPSbuTWXbNQFojSRUuSRojwnPpKJzaCYrqM,QmWJWbHKkKBfPSbuTWXbNQFojSRUuSRojwnPpKJzaCYrqM,61074
8,aww,finelinexcherry,https://styles.redditmedia.com/t5_3mjufk/style...,He really took his time and I respect that,,https://v.redd.it/49yejnttaz691,0,7217,1,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 18:24:24,QmbLwaWhkrL2RJ1auR43kPRRbguVG7t16BHqqu8sUE1VwN,QmbLwaWhkrL2RJ1auR43kPRRbguVG7t16BHqqu8sUE1VwN,55824
9,aww,Chasith,https://styles.redditmedia.com/t5_25343f/style...,This is what cat toes look like on a glass #beans,,https://i.redd.it/wltlzhe8h5791.jpg,0,10469,1,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-22-2022 16:06:54,QmNZmGpL3VKzgs6ztFGYtN6pVsjixQKrCnExrypx2S3Tnb,QmNZmGpL3VKzgs6ztFGYtN6pVsjixQKrCnExrypx2S3Tnb,54708


In [59]:
print(query)


SELECT * FROM post 
JOIN (
    SELECT parent_hash, SUM(num_votes) AS total_votes 
    FROM vote GROUP BY parent_hash
) votes ON votes.parent_hash = post.ipfs_hash
WHERE date(post.date_minted) >= '03-15-2022 17:30:12'  AND post.msa_id_from_query = 1
ORDER BY total_votes DESC
LIMIT 0, 10



In [131]:
for row in cur.execute("PRAGMA table_info('post')"):
    print(row)

(0, 'category', 'STRING', 0, None, 0)
(1, 'username', 'STRING', 0, None, 0)
(2, 'profile_pic', 'STRING', 0, None, 0)
(3, 'title', 'STRING', 0, None, 0)
(4, 'body', 'STRING', 0, None, 0)
(5, 'url', 'STRING', 0, None, 0)
(6, 'is_nsfw', 'NUMERIC', 0, None, 0)
(7, 'block_number', 'INTEGER', 0, None, 0)
(8, 'msa_id_from_query', 'INTEGER', 0, None, 0)
(9, 'signer', 'STRING', 0, None, 0)
(10, 'date_minted', 'DATE', 0, None, 0)
(11, 'ipfs_hash', 'STRING', 0, None, 0)


In [ ]:
for schema, contents in content_jsons.items():
    schema_items = schema.split(',')
    
    for content in contents:
        ipfs_hash = bytes.fromhex(content['data'][2:]).decode()
            
        data = json.loads(client.cat(ipfs_hash).decode())

In [198]:
def subscription_handler(obj, update_nr, subscription_id):
    last_block_updated = update_db(start_block=obj['header']['number'], backfill=False)
    con.commit()
    print(last_block_updated)


result = substrate.subscribe_block_headers(subscription_handler, include_author=False)

post


SubstrateRequestException: {'code': 100, 'message': 'Dispatch Error Module:35 error:2', 'data': ''}

In [164]:
def subscription_handler(obj, update_nr, subscription_id):
    return obj, update_nr, subscription_id
result = substrate.subscribe_block_headers(subscription_handler, include_author=False)

In [211]:
e = substrate.get_events(substrate.get_block_hash(result[0]['header']['number']))

In [218]:
substrate.

'0x36fa31a08a929dd9f08cd57a678cbb877f9d55280db90a727a6d56a14971796f'

In [214]:
e[1].value

{'phase': 'Initialization',
 'extrinsic_idx': None,
 'event': {'event_index': '2300',
  'module_id': 'Messages',
  'event_id': 'MessagesStored',
  'attributes': (159, 8033, 2)},
 'event_index': 35,
 'module_id': 'Messages',
 'event_id': 'MessagesStored',
 'attributes': (159, 8033, 2),
 'topics': []}

In [199]:
result[0]

{'header': {'digest': {'logs': [<scale_info::13(value={'PreRuntime': ('0x61757261', '0xaa8f390800000000')})>,
    <scale_info::13(value={'Seal': ('0x61757261', '0x306ac3ca56ae8e2d4db203c1ae908045547dc039946de605a83d62495ad39f7d6e1eb228029d05b6bccc85d2889fa93b7d2185bb7ffa2dda1f23fe64000a1583')})>]},
  'extrinsicsRoot': '0xe8fec3d67c271817e646e4aab48625c363d84a28317a5a1c5fb432f6e70d541b',
  'number': 8034,
  'parentHash': '0x6f5ac10fd71100e7aadbc3900569a04b9027554370837e130dad29e1bd825751',
  'stateRoot': '0xa8ea03e6d61a4584345f2c12236b4a001f2e7cdc59fb70761fb6f5eb2c659de6'}}

In [203]:
result[0]['header']['digest']['logs'][0]

<scale_info::13(value={'PreRuntime': ('0x61757261', '0xaa8f390800000000')})>

In [31]:
date_str = None
for extrinsic in substrate.get_block(substrate.get_block_hash(2))['extrinsics']:
    if "Timestamp" == extrinsic.value['call']['call_module']:
        timestamp = extrinsic.value['call']['call_args'][0]['value']
        date_time = datetime.fromtimestamp(timestamp/1000)
        date_str = date_time.strftime("%m/%d/%Y, %H:%M:%S")
        break
if date_str is None:
    print('Fail')
date_str

'06/19/2022, 18:16:24'

In [24]:
extrinsic = substrate.get_block(substrate.get_block_hash(2))['extrinsics'][1]

In [26]:
extrinsic.value['call']

{'call_index': '0x0200',
 'call_function': 'set',
 'call_module': 'Timestamp',
 'call_args': [{'name': 'now', 'type': 'Moment', 'value': 1655687784009}],
 'call_hash': '0x9741ab6b11f8e857f9473af31ea267db7e325a03b44840aafff335b68b787b96'}